# COMP 598 - Homework 4

1. Download nyc_311.csv.tgz from MyCourses.
2. Trim it down to only include the incidents that occurred in 2020.

```python
import numpy as np
import pandas as pd

# nyc_311_limit_2.csv is the nyc_311_limit.csv file with header
filename = 'nyc_311_limit_2.csv'

df = pd.read_csv(filename, usecols=[1, 2, 8, 19], low_memory=False)

df_2020 = df[df['Created Date'].str.contains('2020')]

# create csv file
csv_file = df_2020.to_csv('nyc_311_4col.csv', sep=',', index=False)
```

## Task 1

In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
# use the data file you trimmed
def clean_data(filename='nyc_311_4col.csv'):

    df = pd.read_csv(filename)

    # remove rows with missing closed date or incident zip
    df_na = df.dropna()

    # keep rows where status is closed
    df_closed = df_na.loc[df_na['Status'] == 'Closed']

    df_dt = df_closed.copy()

    # convert created date column to datetime format
    df_dt['Created Date'] = pd.to_datetime(df_closed['Created Date'], format='%m/%d/%Y %I:%M:%S %p')

    # convert closed date column to datetime format
    df_dt['Closed Date'] = pd.to_datetime(df_closed['Closed Date'], format='%m/%d/%Y %I:%M:%S %p')

    # if the closed date is before the open date, remove this row
    df_clean = df_dt.loc[df_dt['Created Date'] <= df_dt['Closed Date']]

    return df_clean

def distinct_zipcodes(df, silent=False):
    dzip = pd.unique(df_clean['Incident Zip'])
    
    # print the number of distinct zipcodes in the dataset
    if not silent:
        print(f'Number of distinct zipcodes: {len(dzip)}')
    
    return dzip

def add_month(df):
    df_m = df.copy()
    df_m['Month'] = df['Closed Date'].dt.month
    return df_m

def add_responsetime(df):
    df_rt = df.copy()
    df_rt['Response Time'] = (df['Closed Date'] - df['Created Date']).dt.total_seconds()/3600
    return df_rt

# given month
# all zipcodes
# compute average response time for that month
def avg_rt_all(df, month):
    df_all = df.loc[df['Month'] == month]
    
    avg_rt = df_all['Response Time'].mean()
    
    return avg_rt

# given zipcode and month
# compute average response time
def avg_rt_zip(df, month, zipcode):
    df_all = df.loc[df['Month'] == month]
    df_zip = df_all.loc[df_all['Incident Zip'] == zipcode]
    
    avg_rt = df_zip['Response Time'].mean()
    
    return avg_rt

In [3]:
filename = 'nyc_311_4col.csv'
df_clean = clean_data(filename)
dzip = distinct_zipcodes(df_clean)
df_m = add_month(df_clean)
df_mrt = add_responsetime(df_m)

dzip_f = []
for d in dzip:
    dzip_f.append(int(d))
    
dzip = dzip_f

Number of distinct zipcodes: 236


In [4]:
months = [1,2,3,4,5,6,7,8,9,10,11,12]

# list of monthly average response time for all 2020 data
all_data = []
for m in months:
    value = avg_rt_all(df_mrt, m)
    if not np.isnan(value):
        all_data.append(value)
    else:
        all_data.append(0)
    
# dictionary for 2020 data by zipcode
# key = zipcode
# value = list of monthly average response time
zipcode_data = {}
for d in dzip:
    monthly_rt = []
    
    for m in months:
        value = avg_rt_zip(df_mrt, m, d)
        if not np.isnan(value):
            monthly_rt.append(value)
        else:
            monthly_rt.append(0)
        
    zipcode_data[d] = monthly_rt

In [5]:
data = {}
data['all_data'] = all_data
data['zipcode_data'] = zipcode_data

dzip_sorted = sorted(dzip)
dzip_str = []
for d in dzip_sorted:
    dzip_str.append(str(d))

data['zipcodes'] = dzip_str

with open('data.json', 'w') as json_file:
    json.dump(data, json_file, indent=2)